In [1]:
!wget https://github.com/facebookresearch/fastText/archive/v0.2.0.zip
!unzip v0.2.0.zip
%cd fastText-0.2.0
!make

--2019-08-08 05:17:28--  https://github.com/facebookresearch/fastText/archive/v0.2.0.zip
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.2.0 [following]
--2019-08-08 05:17:29--  https://codeload.github.com/facebookresearch/fastText/zip/v0.2.0
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.2.0.zip’

v0.2.0.zip              [     <=>            ]   4.10M  2.76MB/s    in 1.5s    

2019-08-08 05:17:32 (2.76 MB/s) - ‘v0.2.0.zip’ saved [4304799]

Archive:  v0.2.0.zip
7842495a4d64c7a3bb4339d45d6e64321d002ed8
   creating: fastText-0.2.0/
   creating: fastText-0.2.0/.circleci/
  inflating: fast

In [2]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 3309 (delta 0), reused 8 (delta 0), pack-reused 3298
Receiving objects: 100% (3309/3309), 7.92 MiB | 5.36 MiB/s, done.
Resolving deltas: 100% (2070/2070), done.
/content/fastText-0.2.0/fastText
Processing /content/fastText-0.2.0/fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2380885 sha256=da418fb1eee91199fc1f1c46f5f49f98b453fa0492c68cde1674613249edb117
  Stored in directory: /tmp/pip-ephem-wheel-cache-b4nupa3_/wheels/bd/b4/45/ba7d23789b3bf6bb213208004c2aa23a6405031a0c99a19e1e
Successfully built fasttext


Import training data fron Kaggle

In [0]:
#!pip install kaggle
#!mkdir /root/.kaggle

#!ls /root/.kaggle

#open('/root/.kaggle/kaggle.json','w').write('{"username":"vaqasam","key":"2ec5d91c66a678bc314920e84c687f3b"}')
#!chmod 600 /root/.kaggle/kaggle.json

#%cd /content
#!kaggle datasets download kazanova/sentiment140



Import training data from your personal Google Drive

In [0]:
# Import from drive
#https://drive.google.com/open?id=1T_DTe-7QzGj-_c4Nuv_spWBpPMyRaToU
  
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1T_DTe-7QzGj-_c4Nuv_spWBpPMyRaToU' #@param {type:"string"}

fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('model_tweet.bin')  
# df3 = pd.read_csv('Filename.csv')
# Dataset is now stored in a Pandas

In [5]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [0]:
import csv  
import re

train = open('tweets.train','w')  
test = open('tweets.valid','w')  
with open('training.1600000.processed.noemoticon.csv', mode='r', encoding = "ISO-8859-1") as csv_file:  
    csv_reader = csv.DictReader(csv_file, fieldnames=['target', 'id', 'date', 'flag', 'user', 'text'])
    line = 0
    for row in csv_reader:
        # Clean the training data
        # First we lower case the text
        text = row["text"].lower()
        # remove links
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
        #Remove usernames
        text = re.sub('@[^\s]+','', text)
        # replace hashtags by just words
        text = re.sub(r'#([^\s]+)', r'\1', text)
        #correct all multiple white spaces to a single white space
        text = re.sub('[\s]+', ' ', text)
        # Additional clean up : removing words less than 3 chars, and remove space at the beginning and teh end
        text = re.sub(r'\W*\b\w{1,3}\b', '', text)
        text = text.strip()
        line = line + 1
        # Split data into train and validation
        if line%16 == 0:
            print(f'__label__{row["target"]} {text}', file=test)
        else:
            print(f'__label__{row["target"]} {text}', file=train)

In [7]:
!./fastText-0.2.0/fasttext supervised -input tweets.train -output model_tweet

Read 13M words
Number of words:  578539
Number of labels: 2
Progress: 100.0% words/sec/thread:  358499 lr:  0.000000 loss:  0.328418 ETA:   0h 0m


In [8]:
!pip install fasttext

__label__0 = bad, controversial, mixed,

__label__4 = good

In [37]:
import fasttext

classifier = fasttext.load_model("/content/model_tweet.bin")  
texts = "this sucks"

labels = classifier.predict(texts)  
print (labels[0][0])  

__label__0


In case you want to save the model to your drive:

In [0]:
# Save to drive
#from google.colab import drive
#drive.mount('/content/gdrive')
#!cp /content/model_tweet.bin "/content/gdrive/My Drive/model_tweet.bin"